In [4]:
import tensorflow as tf
import math

In [6]:
math.log(1/0.5)

0.6931471805599453

In [7]:
label_lst = [1 for _ in range(4)]

In [12]:
name_str = "img_321"

In [15]:
type(name_str.encode(encoding='utf-8'))

bytes

In [14]:
name_str.encode(encoding='utf-8')

b'img_321'

In [32]:
range(1, 6)[0]

1